In [ ]:
import pandas as pd
import altair as alt

In [ ]:
ambry = '/Users/ivan/Downloads/Ambry_BARD1.xlsx'
sge = '../Data/20250122_BARD1_SGEscores_wAAsub.xlsx'
gnomad = '../Data/20240905_BARD1_gnomADv4.1.0_SNVs.xlsx'

#Functional Class Cutoffs
path_max = 0.689682159032362 
benign_min = 0.807231141721117

In [ ]:
def read_data(ambry, gnomad, sge):
    ambry = pd.read_excel(ambry)
    ambry = ambry[['Gene', 'c_variant', 'p_variant', 'Classification']]

    
    sge = pd.read_excel(sge)
    sge = sge[['chrom', 'target','AAsub', 'pos_id','Consequence', 'snv_score']]

    
    gnomad = pd.read_excel(gnomad)
    gnomad = gnomad[['Chromosome', 'Position', 'Reference', 'Alternate', 'Allele Frequency']]

    return ambry, sge, gnomad

In [ ]:
def classify_vars(sge, path_max, benign_min):
    sge['Classification'] = None

    sge.loc[sge['snv_score'] <= path_max, 'Classification'] = 'NF'
    sge.loc[sge['snv_score'] >= benign_min, 'Classification'] = 'F'
    sge.loc[sge['Classification'] == None, 'Classification'] = 'I'
    
    return sge

In [ ]:
def process_gnomad(gnomad):
    gnomad['Position'] = gnomad['Position'].astype(str)
    gnomad['pos_id'] = gnomad['Position'] + ':' + gnomad['Alternate']

    return gnomad

In [ ]:
def process_ambry(ambry):
    print(ambry)

In [ ]:
def get_oddsratio(sge, gnomad, ambry):
    sge_nf = sge.loc[sge['Classification'].isin(['NF'])]
    total_gnomad = len(gnomad)

In [ ]:
def main():
    ambry_data, sge_data, gnomad_data = read_data(ambry, gnomad, sge)
    sge_classified = classify_vars(sge_data, path_max, benign_min)
    gnomad_data = process_gnomad(gnomad_data)
    process_ambry(ambry_data)
    get_oddsratio(sge_classified, gnomad_data, ambry_data)

In [ ]:
main()